In [24]:
from torch import FloatTensor
from torch import LongTensor 
from Linear import *

In [25]:
class Module(object):
    def forward(self, *input): 
        raise NotImplementedError
    def backward(self, *gradwrtoutput): 
        raise NotImplementedError
    def param(self): 
        return []

In [26]:
epsilon = 1

In [27]:
class Linear (Module):
    def __init__(self, in_features, out_features, init_w, init_b):
        super(Module, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.w = init_w
        self.b = init_b #TODO maybe put some specific parameters for init, 
                                                            #try xavier
        self.dl_dw = FloatTensor(self.w.size())
        self.dl_db = FloatTensor(self.b.size())  
    def forward(self, x_in):
        x_out = x_in.mm(self.w.t())+self.b
        self.x_in = x_in
        return x_out
    def backward(self, dl_dx_out): 
        dl_dw = dl_dx_out.t().mm(self.x_in)
        dl_db = dl_dx_out.sum(0).view(1,-1)
        dl_dx_in = dl_dx_out.mm(self.w)
        self.dl_dw = dl_dw
        self.dl_db= dl_db
        return dl_dx_in
    def param(self): 
        return [(self.w, self.dl_dw), (self.b, self.dl_db)]

In [28]:
in_features = 3
out_features = 5

input_tensor = FloatTensor(10, 3).normal_()
output_tensor = FloatTensor(10, 5).normal_()

init_w = FloatTensor(out_features, in_features).normal_() 
init_b = FloatTensor(1, out_features).normal_() 
init=[init_w, init_b]

In [29]:
from torch import nn
from torch.autograd import Variable
model = Linear(in_features, out_features, init_w, init_b)
module = nn.Linear(3, 5)
for i, param in enumerate(module.parameters()):
    param.data=init[i]
for i, param in enumerate(module.parameters()):
    print(param)
for param in model.param():
    print(param[0])

TypeError: __init__() takes 3 positional arguments but 5 were given

In [22]:
model.forward(input_tensor)
model.backward(output_tensor)


 0.2590 -3.6839  2.4300
 1.3976 -2.5119 -0.1596
-0.0211 -2.2905  0.9514
 1.2572 -5.6969  6.6645
-0.1773  2.1140  0.5697
 0.4727 -2.5847  2.2213
 0.2784  1.3090 -2.2239
 0.4803 -1.3219  0.7033
-0.1751  2.4865 -0.3963
 0.0721  1.8738 -3.9425
[torch.FloatTensor of size 10x3]

In [23]:
input_var = Variable(input_tensor, requires_grad=True)
forw = module(input_var)
forw.backward(output_tensor)
#input_var.grad.data
for i, param in enumerate(module.parameters()):
    print(param.grad.data)
for param in model.param():
    print(param[1])


-2.6776 -6.6596 -5.2650
 0.9843 -1.1908  0.2481
 2.5419 -0.2323 -4.3181
-1.3027  0.7520 -1.8173
 0.3432  0.8754  4.0333
[torch.FloatTensor of size 5x3]


 5.5675  3.4558 -2.8613 -0.6915  3.8911
[torch.FloatTensor of size 1x5]


-2.6776 -6.6596 -5.2650
 0.9843 -1.1908  0.2481
 2.5419 -0.2323 -4.3181
-1.3027  0.7520 -1.8173
 0.3432  0.8754  4.0333
[torch.FloatTensor of size 5x3]


 5.5675  3.4558 -2.8613 -0.6915  3.8911
[torch.FloatTensor of size 1x5]

